In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:\\Users\\Smruti Deshpande\\Desktop\\Projects\\Datathon_kj\\elearning_churn_dataset.csv")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Handle Missing Values
# For numerical columns, we fill with median or mean based on the nature of the column
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))

# For categorical columns, we fill missing values using forward fill or backward fill
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(method='ffill'))

# Handle Duplicates
df = df.drop_duplicates()

# Feature Engineering
# Engagement Score Calculation
df['Engagement Score'] = df['Login Frequency (per month)'] * df['Time Spent per Session (mins)']

# Encode Categorical Features
encoder = LabelEncoder()
df['Subscription Plan'] = encoder.fit_transform(df['Subscription Plan'])
df['Feedback Sentiment'] = encoder.fit_transform(df['Feedback Sentiment'])
df['Support Ticket Sentiment'] = encoder.fit_transform(df['Support Ticket Sentiment'])
df['Course Reminder Response'] = encoder.fit_transform(df['Course Reminder Response'])
df['Gamified Progress Engagement'] = encoder.fit_transform(df['Gamified Progress Engagement'])

# Convert Date columns to datetime
df['Enrollment Date'] = pd.to_datetime(df['Enrollment Date'], format='%d-%m-%Y')
df['Last Active Date'] = pd.to_datetime(df['Last Active Date'], format='%d-%m-%Y')

# Calculate the days since last active
df['Days Since Last Active'] = (pd.to_datetime('today') - df['Last Active Date']).dt.days

# Normalize numerical features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical
_cols])

# Separate Independent Variables and Target Variable
X = df.drop(['churn'], axis=1)  # Independent features
y = df['churn']  # Target variable


C:\Users\Smruti Deshpande\AppData\Local\Temp\ipykernel_15904\3177323329.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(method='ffill'))


In [3]:
%pip install torch 

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -U "accelerate>=0.26.0"


Note: you may need to restart the kernel to use updated packages.


In [33]:
print("Dataset Shape:", df.shape)  # Should not be (0, N)
print(df.head())  # Display first few rows
print(df.columns)  # Ensure correct column names


Dataset Shape: (1000, 23)
  User ID Enrollment Date  Subscription Plan  Course Completion Rate  \
0   U0001      2022-04-13                  0               -0.448101   
1   U0002      2023-03-12                  0               -0.373640   
2   U0003      2022-09-28                  2                0.235746   
3   U0004      2022-04-17                  2               -1.040369   
4   U0005      2022-03-13                  2               -0.971899   

   Courses Enrolled  Login Frequency (per month)  \
0          1.531276                    -0.770452   
1         -1.560650                     0.698708   
2          0.758295                     1.310858   
3          0.758295                     0.576278   
4         -0.401177                    -1.015312   

   Time Spent per Session (mins) Last Active Date  Forum Participation  \
0                       1.765516       2023-04-04            -0.790448   
1                       0.317900       2023-11-22            -0.931235   
2     

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
import torch
import pandas as pd
import os

# Check the shape of the initial dataset
print("Original dataset shape:", df.shape)

# Ensure NA values in 'Support Ticket Sentiment' are replaced with 'neutral'
sentiment_data = df[['Feedback Sentiment', 'Support Ticket Sentiment']].copy()
sentiment_data['Support Ticket Sentiment'] = sentiment_data['Support Ticket Sentiment'].fillna('neutral')

# Check unique label values to ensure proper mapping
print("Unique label values before processing:", sentiment_data['Support Ticket Sentiment'].unique())

# Combine relevant features for sentiment analysis
sentiment_data['text'] = sentiment_data['Feedback Sentiment'].astype(str) + " " + sentiment_data['Support Ticket Sentiment'].astype(str)

# Directly use numeric labels (assuming they're already in 0, 1, 2, 3 format)
sentiment_data['label'] = sentiment_data['Support Ticket Sentiment']

# Ensure no NaN values in the dataset
sentiment_data = sentiment_data.dropna(subset=['label'])

# Convert labels to integers if needed
sentiment_data['label'] = sentiment_data['label'].astype(int)

# Check the dataset after cleaning
print("Dataset shape after cleaning:", sentiment_data.shape)
print("Unique label values after processing:", sentiment_data['label'].unique())

# Check for class imbalance
print(sentiment_data['label'].value_counts(normalize=True))

# Apply Random Oversampling if there is class imbalance
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = ros.fit_resample(sentiment_data[['text']], sentiment_data['label'])

# Ensure there's data left for splitting
if len(X_resampled) == 0:
    print("The dataset is empty after resampling. Exiting process.")
    exit()

# Increase test set size to ensure diverse validation set
X_sentiment_train, X_sentiment_test, y_sentiment_train, y_sentiment_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42  # Increased test size
)

# Verify the number of samples in each split
print("Number of training samples:", len(X_sentiment_train))
print("Number of testing samples:", len(X_sentiment_test))

# Tokenization using BERT
# Tokenization using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode text data with truncation and padding
train_encodings = tokenizer.batch_encode_plus(
    X_sentiment_train['text'].tolist(),  # Use .tolist() on the 'text' column
    padding="max_length",
    truncation=True,
    max_length=128,  # Ensure appropriate input length
    return_tensors="pt"
)

test_encodings = tokenizer.batch_encode_plus(
    X_sentiment_test['text'].tolist(),  # Use .tolist() on the 'text' column
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)


# Convert the data into PyTorch datasets
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset instances
train_dataset = SentimentDataset(train_encodings, y_sentiment_train)
test_dataset = SentimentDataset(test_encodings, y_sentiment_test)

# Use BERT with increased dropout regularization
config = BertConfig.from_pretrained(
    'bert-base-uncased',
    num_labels=4,
    hidden_dropout_prob=0.3,  # Increased dropout to reduce overfitting
    attention_probs_dropout_prob=0.3
)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

# Training Arguments with Early Stopping and Learning Rate Scheduler
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Reduce epochs to prevent overfitting
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",
    per_device_train_batch_size=32,  # Increased batch size for better stability
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,  # Simulate larger batch sizes
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-5,  # Reduced learning rate
    lr_scheduler_type="linear",  # Gradually reduce learning rate
    load_best_model_at_end=True,  # Load best model based on validation loss
    metric_for_best_model="accuracy",
    save_total_limit=2,  # Keep only the best 2 models
)

# Early stopping callback
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)  # Stop if no improvement in 3 eval steps

# Custom compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions)  # Convert predictions to a tensor
    predictions = torch.argmax(predictions, dim=-1)  # Convert logits to predicted class
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}


# Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Pass the custom metric function
    callbacks=[early_stopping]  # Add early stopping
)

# Train the model
trainer.train()

# Evaluate the model
eval_result = trainer.evaluate()
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")


Original dataset shape: (1000, 23)
Unique label values before processing: [3 0 2 1]
Dataset shape after cleaning: (1000, 4)
Unique label values after processing: [3 0 2 1]
label
1    0.430
2    0.424
0    0.145
3    0.001
Name: proportion, dtype: float64
Number of training samples: 1204
Number of testing samples: 516


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Smruti Deshpande\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.370700,1.031580,0.705426
2,1.212100,0.913807,0.759690
3,1.009200,0.682088,0.744186
4,0.826100,0.504846,0.891473
5,0.706000,0.459946,0.875969


Accuracy: 0.8915


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import joblib
from sklearn.metrics import accuracy_score

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = joblib.load('./results/tokenizer.joblib')

# Test data
test_texts = ["The product was great, I had an amazing experience!", "The service was terrible, I am very disappointed."]
test_labels = [1, 0]  # Example test labels

# Tokenize the test data
test_encodings = tokenizer.batch_encode_plus(
    test_texts,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Create a PyTorch dataset for evaluation
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create test dataset
test_dataset = SentimentDataset(test_encodings, test_labels)

# Define evaluation function
def evaluate(model, test_dataset):
    model.eval()
    predictions = []
    true_labels = []
    for batch in torch.utils.data.DataLoader(test_dataset, batch_size=32):
        inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(model.device)
        true_labels.extend(labels.cpu().numpy())

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            predictions.extend(preds)

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy:.4f}")

# Evaluate the model on the test data
evaluate(model, test_dataset)
